In [25]:
#import required modules
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import random
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import glob

#name of the model
NAME = "CATDOG"
run=6
#path of datsets 
DATADIR = "/Users/jainparag/Desktop/CatsvsDogs_classifier/PetImages"

CATEGORIES = ["Dog", "Cat"]

#resizing all images 

IMG_SIZE = 150


In [2]:
#creating training dataset
training_data = []
def create_training_data():
    
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  #read and convert image in grayscale
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

create_training_data()

print(len(training_data))


100%|██████████| 12502/12502 [00:24<00:00, 508.45it/s]

24518


In [3]:
#shuffling the data
random.shuffle(training_data)

In [4]:
#creating a list of features and labels
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


In [5]:
#creating the model 
X = np.array(X/255.0) #normalizing the images
y=np.array(y)

model = Sequential() #defining the type of model

# creating a hideen layer
model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:])) #creating convolution layers
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64)) #fully connected layer
model.add(Activation('relu'))

model.add(Dense(1)) #output layer
model.add(Activation('sigmoid'))

tensorboard = TensorBoard(log_dir="logs/{}_{}".format(NAME, run)) #training logs

#compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

#training the model
model.fit(X, y,
          batch_size=32,
          epochs=3,
          validation_split=0.3,
          callbacks=[tensorboard])

Epoch 1/3
537/537 [==============================] - 299s 556ms/step - loss: 0.6532 - accuracy: 0.6203 - val_loss: 0.6010 - val_accuracy: 0.6827
Epoch 2/3
537/537 [==============================] - 283s 527ms/step - loss: 0.5531 - accuracy: 0.7224 - val_loss: 0.5360 - val_accuracy: 0.7369
Epoch 3/3
537/537 [==============================] - 282s 526ms/step - loss: 0.4461 - accuracy: 0.7923 - val_loss: 0.5146 - val_accuracy: 0.7535


In [31]:
# prediction

# pre processing the test images
def prepare(filepath):  
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


new = tf.keras.models.load_model("{}_run.model".format(NAME)) #loading the model


path=glob.glob("/Users/jainparag/Desktop/CatsvsDogs_classifier/Testimages/*")
for i in path:
    print(os.path.split(i)[1])
    prediction = new.predict([prepare(i)])#predicting the label of test images
    print(CATEGORIES[int(prediction[0][0])]) # displaying the label of the test images

8.JPG
Dog
9.JPG
Dog
4.jpg
Dog
5.jpg
Cat
7.JPG
Cat
6.JPG
Cat
2.JPG
Dog
3.JPG
Dog
1.JPG
Dog
